In [1]:
import numpy as np
# import seaborn as sns
from matplotlib import pyplot as plt
import math
import copy
import random
from scipy.spatial import distance
import operator
import sys
import math

In [2]:
# VRP1
# import distance matrix
filepath = 'VRP1/distance.txt'
with open(filepath, 'r') as t:
    distance_matrix = []
    for line in t.readlines():
        y = [int(value) for value in line.split()]
        distance_matrix.append(y)
        
    t.close()
    distance_matrix = np.array(distance_matrix)

# import demand matrix
filepath = 'VRP1/demand.txt'
with open(filepath, 'r') as t:
    demand_matrix = []
    for line in t.readlines():
        y = [int(value) for value in line.split()]
        demand_matrix.append(y)
        
    t.close()
    demand_matrix = np.array(demand_matrix)
    demand_matrix = demand_matrix[0]
    # since demand matrix has length 100 but the distance_matrix has length 101, we add one demand with 0 to represent the depot 
    demand_matrix = np.insert(demand_matrix,0,0)

# import capacity matrix
filepath = 'VRP1/capacity.txt'
with open(filepath, 'r') as t:
    capacity_matrix = []
    for line in t.readlines():
        y = [int(value) for value in line.split()]
        capacity_matrix.append(y)
        
    t.close()
    capacity_matrix = np.array(capacity_matrix)
    capacity_matrix = capacity_matrix[0]

# import transportation_cost matrix
filepath = 'VRP1/transportation_cost.txt'
with open(filepath, 'r') as t:
    transportation_cost = []
    for line in t.readlines():
        y = [int(value) for value in line.split()]
        transportation_cost.append(y)
        
    t.close()
    transportation_cost = np.array(transportation_cost)
    transportation_cost = transportation_cost[0]
# VRP2
# import distance matrix
filepath = 'VRP2/distance.txt'
with open(filepath, 'r') as t:
    distance_matrix_2 = []
    for line in t.readlines():
        y = [int(value) for value in line.split()]
        distance_matrix_2.append(y)
        
    t.close()
    distance_matrix_2 = np.array(distance_matrix_2)

# import demand matrix
filepath = 'VRP2/demand.txt'
with open(filepath, 'r') as t:
    demand_matrix_2 = []
    for line in t.readlines():
        y = [int(value) for value in line.split()]
        demand_matrix_2.append(y)
        
    t.close()
    demand_matrix_2 = np.array(demand_matrix)
    demand_matrix_2 = demand_matrix_2[0]
    # since demand matrix has length 100 but the distance_matrix has length 101, we add one demand with 0 to represent the depot 
    demand_matrix_2 = np.insert(demand_matrix,0,0)

    
# import capacity matrix
filepath = 'VRP2/capacity.txt'
with open(filepath, 'r') as t:
    capacity_matrix_2 = []
    for line in t.readlines():
        y = [int(value) for value in line.split()]
        capacity_matrix_2.append(y)
        
    t.close()
    capacity_matrix_2 = np.array(capacity_matrix_2)
    capacity_matrix_2 = capacity_matrix_2[0]
    

    
# import transportation_cost matrix
filepath = 'VRP2/transportation_cost.txt'
with open(filepath, 'r') as t:
    transportation_cost_2 = []
    for line in t.readlines():
        y = [int(value) for value in line.split()]
        transportation_cost_2.append(y)
        
    t.close()
    transportation_cost_2 = np.array(transportation_cost_2)
    transportation_cost_2 = transportation_cost_2[0]

In [3]:
def average_dist(cluster):
    
    sum_distance = 0
    avg_distance = 0
    
    # from every node to every node in the current cluster calculate the overall distance
    for node1 in cluster:
        for node2 in cluster:
            sum_distance += distance_matrix[node1][node2]

    # and calculate the average as an approximation
    one_edge = sum_distance/(len(cluster)**2-2)
    avg_distance = one_edge * (len(cluster)-1)
    
    return avg_distance

In [4]:
def total_demand_satisfied(route):
    total_demand_satisfied = 0
    for i in route:
        total_demand_satisfied += demand_matrix[i]
    return total_demand_satisfied

In [5]:
def total_cars():
    '''
    returns the sorted sets of capacities available with their according costs and those zipped information as dict
    '''
    unique_capacity = sorted(set(capacity_matrix)) #VRP1: 100, 300, 500, 1000
    unique_cost = sorted(set(transportation_cost)) #VRP1: 11, 13, 15, 19
    
    cap = capacity_matrix
    num_cars = [list(cap).count(value) for value in unique_capacity]
    total_cars = dict(zip(unique_capacity, num_cars))
    
    return unique_capacity, unique_cost, total_cars

In [6]:
global cars_left
_, _, cars_left = total_cars()
global cars_used
cars_used = {}
print(cars_left)

{100: 20, 300: 7, 500: 4, 1000: 2}


In [7]:
def optimal_car(single_cluster):
    '''
    check the output of the function with
        if type(x) == str:
    if it's True, then there is no car available for this cluster
    '''
    global cars_left
    
    unique_capacity, unique_cost, _ = total_cars()
    
    copy_cars_left = cars_left
    cars_possible = []
    cars_available = []
    
    for value in range(len(unique_capacity)):
        if total_demand_satisfied(single_cluster) < unique_capacity[value]:
            cars_possible.append(unique_capacity[value])
    
    cars_possible = sorted(cars_possible)
    cars_available = cars_possible[:]
    
    for car in cars_possible:
        test_value = copy_cars_left[car] - 1
        if test_value < 0:
            cars_available.remove(car)
            
    try:
        optimal_car = min(cars_available)
        return optimal_car
        
    except ValueError:
        return "no car available"

In [8]:
def update_cars_left(string, used_car):
    '''
    takes as input the chosen car and removes it from the global variable cars_left
    '''
    global cars_left
    
    if string == "+":
        cars_left[used_car] += 1
    if string == "-":
        cars_left[used_car] -= 1

In [9]:
def cluster_initialization():

    global cars_left
    global cars_used
    
    list_of_clusters = []
    customers = list(range(len(demand_matrix))[1:-1])
    visited_customers = []
    remaining_distance_matrix = copy.deepcopy(distance_matrix[0])

    while len(visited_customers) < len(customers)-1: 
        cluster = []
        demand = 0

        #add customers with smallest demand/distance ratio to cluster as long as overall demand of cluster is <= 100
        while total_demand_satisfied(cluster) < min(capacity_matrix):
            best_demand_distance_ratio = 0
            
            if cluster == []:
                #add depot, because that's where we are starting
                cluster.append(0)
                #choose customer furthest away from depot to create cluster
                while(cluster == [0]):
                    if visited_customers == []:
                        pass
                    else:
                        for values in visited_customers:
                            remaining_distance_matrix[values] = 0
                    first_customer = np.argmax(remaining_distance_matrix)
                    #add the best fitting customer to the cluster 
                    if first_customer not in visited_customers:
                        visited_customers.append(first_customer)
                        cluster.append(first_customer)

            else:
                #go through all customers
                for i in range(len(customers)):

                    #if customer is not added yet
                    if customers[i] not in visited_customers:
                        
                        #create a hypothetical cluster with the regarded customer and check the resulting demand-distance-ratio
                        cluster_copy = cluster[:]
                        cluster_copy.append(customers[i])
                        cluster_copy.append(0)

                        #calculate the demand_distance_ratio for comparison
                        demand_distance_ratio = total_demand_satisfied(cluster_copy) / average_dist(cluster_copy)

                        #find best customer
                        if demand_distance_ratio > best_demand_distance_ratio and total_demand_satisfied(cluster_copy) <= min(capacity_matrix):
                            best_demand_distance_ratio = demand_distance_ratio
                            best_customer = customers[i]

                if best_customer in visited_customers:
                    break
                #add the best fitting customer to the cluster
                else:
                    cluster.append(best_customer)
                    visited_customers.append(best_customer)
        cluster.append(0)
        update_cars_left("-", min(capacity_matrix))
        
        cars_used[str(cluster)] = min(capacity_matrix)
        
        list_of_clusters.append(cluster)
        
    return np.asarray([np.array(clusters) for clusters in list_of_clusters])

In [10]:
# has only one cluster as input
def demand_cost_ratio(cluster, capacity):
    #RETURN INDEX NOT VALUE
    #when using, cast the yield into a list and get max --> max(list(yield))
    cluster = list(cluster)
    unique_capacity = sorted(set(capacity_matrix))
    unique_cost = sorted(set(transportation_cost))

    capacity_dict = dict(zip(unique_capacity, unique_cost))
    cost = capacity_dict.get(capacity)

    
    demand_cost_ratio = total_demand_satisfied(cluster)/(average_dist(cluster) * cost) 
    
    return demand_cost_ratio

In [11]:
def evaluate_solution(list_of_clusters):
    
    global cars_used
    total_cost = 0
   
    unique_capacity, unique_cost, _ = total_cars()
    
    cost_dict = dict(zip(unique_capacity, unique_cost))
    sum_distance = 0
    
    for cluster in list_of_clusters:
        capacity = cars_used.get(str(cluster))
        cost = cost_dict.get(capacity)
        sum_distance = average_dist(cluster)
        
#         for node in range(len(cluster)-1):
#             sum_distance += distance_matrix[node][node+1]
        
        total_cost += (cost*sum_distance)
        
    return total_cost

In [12]:
def cost(cluster):
    
    cost = 0
    total_cost = 0
    
    unique_capacity, unique_cost, _ = total_cars()
    cost_dict = dict(zip(unique_capacity, unique_cost))
    
    capacity = cars_used.get(str(cluster))
    if capacity == None:
        capacity = optimal_car(cluster)
        
#     print("cluster", cluster)
#     print("capacity", capacity)
#     print("cars_used", cars_used)
    cost = cost_dict.get(capacity)
#     print("cost_dict",cost_dict)
    sum_distance = average_dist(cluster)
    
#     print("cost", cost)
#     print("sum_distance", sum_distance)
    total_cost = cost*sum_distance
    
    
    return total_cost

In [13]:
def merge(initial_clusters):
    
    '''
    returns the merged clusters 
    '''
    
    global cars_used
    
    initial_cost_ratio = []
    
    for cluster in initial_clusters:
        initial_cost_ratio.append(demand_cost_ratio(cluster, optimal_car(cluster)))
        
    solution = [list(value) for value in initial_clusters]
    
    #sort clusters according to demand/ cost ratio
    cluster_and_cost_dict = dict( zip(initial_cost_ratio, solution))
    sorted_cluster_and_cost_dict = sorted(cluster_and_cost_dict.items(), key = lambda s: s[0])
    
    #first approach was to only consider the 50% worst initial clusters
    #but yields worse results; if you want to try this also uncomment the join with the other 50% before the return
    #this variable mainly tells how around many 100 capacity cars are kept(i.e not selected for the merge in this iteration)
    #split_point = math.floor(len(sorted_cluster_and_cost_dict)/2)
    
    # consindering all initial clusters   
    clusters_to_be_merged = []
    for i in range(len(solution)):
        clusters_to_be_merged.append(sorted_cluster_and_cost_dict[i][1])
    
    
    copy_clusters_to_be_merged = clusters_to_be_merged[:]
    print(copy_clusters_to_be_merged
         )
    start_cluster = []
    next_cluster = []
    new_cluster = []
    merged_clusters = []
    new_solution = []

    i = 0
    j = 0
        
    len_initial_cluster = len(clusters_to_be_merged)
    
    while j < len_initial_cluster:
        start_cluster = clusters_to_be_merged[j]
        i = j
    
        while i < len_initial_cluster-1:
            next_cluster = clusters_to_be_merged[i+1]
            merged_clusters = start_cluster[:-1] + next_cluster[1:]

            if new_solution == []: 
                new_solution.append(start_cluster)
            if all([next_cluster not in new_solution]): 
                new_solution.append(next_cluster)
            
            # from the global variable cars_used
            start_car = cars_used.get(str(start_cluster))
            next_car = cars_used.get(str(next_cluster))
            
            # checks if the optimal car for this cluster is available (if not, then merging is not possible)
            if optimal_car(merged_clusters) == "no car available":
                break
            else:
                merged_car = optimal_car(merged_clusters)

            # that was a different approach, substituting the demand_cost_ratio and using only cost() in the next if-statement
            #cost_factor_ratio = ((start_car + next_car)/merged_car)
            
            #print("start_car:", start_car)
            #print("next_car:", next_car)
            #print("merged_cluster", total_demand_satisfied(merged_clusters))
            #print("merged_car", merged_car)           

            #if the merged_cluster has a better demand_cost_ratio, merge
            #if (demand_cost_ratio(start_cluster, start_car)) <= (demand_cost_ratio(merged_clusters, merged_car)):
            
            #different if statement that considers only cost not; unfortunately: IT'S NOT MERGING
            if (cost(start_cluster)) >= (cost(merged_clusters)):
                
                index = new_solution.index(start_cluster)
                new_solution[index] = merged_clusters[:]
                new_solution.remove(next_cluster)

                del cars_used[str(start_cluster)]
                start_cluster = merged_clusters[:]

                len_initial_cluster = len_initial_cluster - 1
                
                # This is an annoying case. Happens when new_solution's last element is the merged cluster(else statement are the usual steps while if and elif take care of the boundary cases)   
                if len(new_solution) == 1:
                    clusters_to_be_merged = new_solution[:] + clusters_to_be_merged[1:]
                elif new_solution[-1] not in clusters_to_be_merged:
                    index_end = clusters_to_be_merged.index(new_solution[-2])
                    clusters_to_be_merged = new_solution[:] + clusters_to_be_merged[index_end+2:]
                else:
                    index_end = clusters_to_be_merged.index(new_solution[-1])
                    clusters_to_be_merged = new_solution[:] + clusters_to_be_merged[index_end+1:]


                if next_cluster in clusters_to_be_merged:
                    clusters_to_be_merged.remove(next_cluster)

                # update global variable cars_left used for function optimal_car()
                update_cars_left("+", start_car)
                update_cars_left("+", next_car)
                update_cars_left("-", merged_car)

                del cars_used[str(next_cluster)]
                cars_used[str(merged_clusters)] = merged_car

                # update while loops
                i = clusters_to_be_merged.index(start_cluster) - 1
                len_initial_cluster = len(clusters_to_be_merged)

             
            i = i + 1
        j = j + 1
        
    better_solution = clusters_to_be_merged[:]
    
    #only when taking the worst clusters
    #for i in range(len(solution) - split_point):
        #better_solution.append(sorted_cluster_and_cost_dict[i+split_point][1])
    
    return better_solution

In [14]:
initial = cluster_initialization()
print("cars_left:", cars_left)
final_solution = merge(initial)

for cluster in final_solution:
    print(total_demand_satisfied(cluster))
    
print("final_solution:", final_solution)
print("len:", len(final_solution))
print("cars_left:", cars_left)
print("total cost:", evaluate_solution(final_solution))

for cluster in final_solution:
    print(total_demand_satisfied(cluster))

cars_left: {100: 0, 300: 7, 500: 4, 1000: 2}
[[0, 47, 50, 60, 88, 78, 74, 0], [0, 30, 92, 80, 34, 90, 93, 95, 0], [0, 6, 7, 15, 91, 94, 53, 69, 0], [0, 44, 38, 20, 52, 19, 0], [0, 11, 16, 27, 41, 36, 40, 98, 0], [0, 51, 65, 14, 5, 25, 0], [0, 29, 21, 79, 45, 83, 0], [0, 75, 32, 17, 56, 0], [0, 48, 89, 22, 28, 61, 8, 0], [0, 71, 18, 82, 70, 57, 0], [0, 73, 37, 55, 87, 0], [0, 99, 77, 63, 23, 0], [0, 12, 35, 46, 62, 0], [0, 59, 64, 43, 24, 54, 0], [0, 4, 49, 42, 86, 0], [0, 3, 10, 76, 1, 13, 0], [0, 96, 84, 72, 97, 0], [0, 33, 39, 26, 67, 0], [0, 68, 9, 58, 81, 0], [0, 85, 31, 2, 66, 0]]
71
98
92
98
99
100
95
92
91
100
99
99
100
95
99
99
97
95
96
94
[[0, 47, 50, 60, 88, 78, 74, 0], [0, 30, 92, 80, 34, 90, 93, 95, 0], [0, 6, 7, 15, 91, 94, 53, 69, 0], [0, 44, 38, 20, 52, 19, 0], [0, 11, 16, 27, 41, 36, 40, 98, 0], [0, 51, 65, 14, 5, 25, 0], [0, 29, 21, 79, 45, 83, 0], [0, 75, 32, 17, 56, 0], [0, 48, 89, 22, 28, 61, 8, 0], [0, 71, 18, 82, 70, 57, 0], [0, 73, 37, 55, 87, 0], [0, 99, 77, 63,

In [15]:
# def keep_merging_until_convergence(initial_clusters):
    
#     cluster1 = merge(initial_clusters)
#     i = 0
#     for i in range(10):
#         i +=1
#         cluster2 = merge(cluster1)
#         cluster1 = merge(cluster2)
        
# #         if i == 0 or i == 99:
# #             print(cluster2)
    
#     return cluster1

In [16]:
# initial = cluster_initialization()
# final_solution = keep_merging_until_convergence(initial)

# for cluster in final_solution:
#     print(total_demand_satisfied(cluster))

# print(cars_left)

In [17]:
# print(evaluate_solution(final_solution))

In [18]:
# def merge(initial_clusters):
    
#     '''
#     returns the merged clusters 
#     '''
    
#     global cars_used
    
#     initial_cost_ratio = []
    
#     for cluster in initial_clusters:
#         initial_cost_ratio.append(demand_cost_ratio(cluster, optimal_car(cluster)))

#     ### initial_clusters instead of initial_solution
#     solution = [list(value) for value in initial_clusters]
    
#     #sort clusters according to demand/ cost ratio
#     cluster_and_cost_dict = dict( zip(initial_cost_ratio, solution))
#     sorted_cluster_and_cost_dict = sorted(cluster_and_cost_dict.items(), key = lambda s: s[0])
    
#     #this variable mainly tells how around many 100 capacity cars are kept(i.e not selected for the merge in this iteration)
# #     split_point = math.floor(len(sorted_cluster_and_cost_dict)/2)
    
#     #Take the top worst clusters    
#     clusters_to_be_merged = []
#     for i in range(len(solution)):
#         clusters_to_be_merged.append(sorted_cluster_and_cost_dict[i][1])
        
#     copy_clusters_to_be_merged = clusters_to_be_merged[:]
    
#     start_cluster = []
#     next_cluster = []
#     new_cluster = []
#     merged_clusters = []
#     new_solution = []

#     i = 0
#     j = 0
        
#     len_initial_cluster = len(clusters_to_be_merged)
    
#     while j < len_initial_cluster:
#         start_cluster = clusters_to_be_merged[j]
#         i = j
    
#         while i < len_initial_cluster-1:
#             next_cluster = clusters_to_be_merged[i+1]
#             merged_clusters = start_cluster[:-1] + next_cluster[1:]

#             if new_solution == []: 
#                 new_solution.append(start_cluster)
#             if all([next_cluster not in new_solution]): 
#                 new_solution.append(next_cluster)
            
#             #if type(optimal_car(merged_clusters)) == int:
#         #try:
#             start_car = cars_used.get(str(start_cluster))
#             #print("start_car:", start_car)
#             next_car = cars_used.get(str(next_cluster))
#             #print("next_car:", next_car)
#             merged_car = optimal_car(merged_clusters)
#             #print("merged_car", merged_car)

#             #bhaskar_ratio = (total_demand_satisfied(start_cluster)+total_demand_satisfied(next_))
#             cost_factor_ratio = ((start_car+next_car)/merged_car)

# #             print("a", sum(cost(start_cluster) , cost(next_cluster)))
# #             print("b", cost(merged_clusters)*cost_ratio)
            
#             if cost_factor_ratio > 1:
# #                 print("sum_of_2_clusters", cost(start_cluster)+ cost(next_cluster))
# #                 print("start_cluster", start_cluster)
# #                 print("next_cluster", next_cluster)
# #                 print("start_car", start_car)
# #                 print("next_car", next_car)
#                 print("merged_cluster", merged_clusters)
#                 print("merged_car", merged_car)

                
# #                 print("sum_of_2_clusters", cost(start_cluster)+ cost(next_cluster))
# #                 print("cost(merged_clusters) " , cost(merged_clusters), "and total with ratio" , (cost(merged_clusters)*cost_factor_ratio))
# #             if demand_cost_ratio(start_cluster, start_car) <= demand_cost_ratio(merged_clusters, merged_car):
#             if (cost(start_cluster) + cost(next_cluster)) >= (cost(merged_clusters)*0.67):
                
#                 index = new_solution.index(start_cluster)
#                 new_solution[index] = merged_clusters[:]
#                 new_solution.remove(next_cluster)

#                 del cars_used[str(start_cluster)]
#                 start_cluster = merged_clusters[:]

#                 #print("new_solution:", new_solution)

#                 len_initial_cluster = len_initial_cluster - 1
#                 index_end = clusters_to_be_merged.index(new_solution[-1])

#                 #update the clusters_to_be_merged in the correct order to avoid duplicate comparisons 

#                 clusters_to_be_merged = new_solution[:] + clusters_to_be_merged[index_end+1:]

#                 if next_cluster in clusters_to_be_merged:
#                     clusters_to_be_merged.remove(next_cluster)

#                 #print("start_car:", start_car)
#                 update_cars_left("+", start_car)
#                 #print("next_car:", next_car)
#                 update_cars_left("+", next_car)
#                 #print("merged_car:", merged_car)
#                 update_cars_left("-", merged_car)


#                 del cars_used[str(next_cluster)]
#                 cars_used[str(merged_clusters)] = merged_car

#                 #print(cars_left)

#                 i = clusters_to_be_merged.index(start_cluster) - 1
#                 len_initial_cluster = len(clusters_to_be_merged)

#         #except TypeError:
#         #    break

                
#             i = i + 1
#         j = j + 1
        
#     better_solution = clusters_to_be_merged[:]
    
#     #for i in range(len(solution) - split_point):
#     #    better_solution.append(sorted_cluster_and_cost_dict[i][1])
#     #print("copy_clusters_to_be_merged",  copy_clusters_to_be_merged)
    
#     return better_solution